In [1]:
%pip install nltk
%pip install keras
%pip install tensorflow
%pip install tensorflow_hub
%pip install transformers
%pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss

2023-10-14 08:27:10.404479: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-14 08:27:12.939922: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-14 08:27:12.939984: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-14 08:27:12.943500: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-14 08:27:13.982260: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-14 08:27:13.983564: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [4]:
# load data set from  stupidstuff.json
df = pd.read_json('stupidstuff.json')
df = df.dropna()
df = df.reset_index(drop=True)
df.head()


,body,category,id,rating
0,A blackjack dealer and a player with a thirtee...,Children,1,2.63
1,"At a dinner party, several of the guests were ...",Blonde Jokes,2,2.57
2,One day this cop pulls over a blonde for speed...,Blonde Jokes,3,3.09
3,Three women are about to be executed for crime...,Blonde Jokes,4,4.10
4,A girl came skipping home FROM school one day....,Blonde Jokes,5,4.30


In [5]:
#print the size of the largest cell in the column body
print(df['body'].map(len).max())


9671


In [6]:
with open('gpt2_training_data.txt', 'w') as file:
    for text in df['body']:
        file.write(text + '\n')

In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Initialize the GPT-2 tokenizer and model
model_name = "gpt2"  # You can choose another variant if needed
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Load your preprocessed dataset
dataset_path = "/workspaces/Joke_Generator/gpt2_training_data.txt"
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=dataset_path,
    block_size=128,  # Adjust the block size as needed
)

# Define a data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # We're doing language modeling, not masked language modeling
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",  # Output directory for checkpoints and logs
    overwrite_output_dir=True,
    per_device_train_batch_size=2,  # Adjust batch size as needed
    num_train_epochs=3,  # Adjust the number of training epochs
    save_steps=10_000,  # Adjust checkpoint save frequency
    save_total_limit=2,  # Limit the number of checkpoints saved
)

# Initialize the Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Start fine-tuning
trainer.train()

# Save the model
trainer.save_model()

# Optionally, you can save the tokenizer and config as well
tokenizer.save_pretrained("./output")
model.config.save_pretrained("./output")


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/codespace/.python/current/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,3.472100
1000,3.353700
1500,3.323500
2000,3.225700
2500,3.019900
3000,2.955400
3500,2.951500
4000,2.948600
4500,2.863000
5000,2.767400


Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.


In [8]:
# Define the directory where you want to save the model and tokenizer
model_directory = "./output"

# Save the model and tokenizer
model.save_pretrained(model_directory)
tokenizer.save_pretrained(model_directory)

print(f"Model and tokenizer saved to {model_directory}")


Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.


Model and tokenizer saved to ./output


In [14]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the saved model and tokenizer
model_directory = "./output"
model = GPT2LMHeadModel.from_pretrained(model_directory)
tokenizer = GPT2Tokenizer.from_pretrained(model_directory)

# Define a prompt
prompt = "arab man"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
response_ids = model.generate(input_ids, max_length=200, num_return_sequences=1)
joke = tokenizer.decode(response_ids[0], skip_special_tokens=True)
print("Joke", joke)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Joke arab man, who was sitting in the back seat, was sitting in the back seat. The man said, "I'm sorry, but I'm blind and can't see." The doctor said, "Well, I'm sure you can see, but you have to be careful with your eyes." The man said, "I'm blind too, and I can't see very well." The doctor said, "Well, I'm sure you can see, but you have to be careful with your eyes." The man said, "I'm sorry, but I'm blind too, and I can't see very well." The doctor said, "Well, I'm sure you can see, but you have to be careful with your eyes." The man said, "I'm sorry, but I'm blind too, and I can't see very well." The doctor said, "Well, I'm sure you can see, but you have to be careful with your eyes." The man said
